<a href="https://colab.research.google.com/github/kenken0830/Graphing/blob/main/diffution_model_read.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Sep 20 04:13:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%cd '/content/drive/My Drive/work'

/content/drive/My Drive/work


In [3]:
!pip install diffusers[training] accelerate transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 153 kB 15.4 MB/s 
     |████████████████████████████████| 143 kB 65.4 MB/s 
     |████████████████████████████████| 4.9 MB 80.7 MB/s 
     |████████████████████████████████| 6.6 MB 64.7 MB/s 
     |████████████████████████████████| 120 kB 101.2 MB/s 
     |████████████████████████████████| 365 kB 82.9 MB/s 
     |████████████████████████████████| 212 kB 77.3 MB/s 
     |████████████████████████████████| 115 kB 79.7 MB/s 
     |████████████████████████████████| 127 kB 99.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [5]:
from huggingface_hub import hf_hub_download
import os

# パラメータ
pretrained_model_name_or_path = "CompVis/stable-diffusion-v1-4" # 事前学習モデル
repo_id_embeds = "sd-concepts-library/hitokomoru-style-nao"  # リポジトリID
downloaded_embedding_folder = "./downloaded_embedding"  # 出力フォルダ

# 学習済みモデルのダウンロード
if not os.path.exists(downloaded_embedding_folder):
  os.mkdir(downloaded_embedding_folder)
embeds_path = hf_hub_download(repo_id=repo_id_embeds, filename="learned_embeds.bin", use_auth_token=True)
token_path = hf_hub_download(repo_id=repo_id_embeds, filename="token_identifier.txt", use_auth_token=True)
!cp $embeds_path $downloaded_embedding_folder
!cp $token_path $downloaded_embedding_folder

Downloading:   0%|          | 0.00/3.82k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

In [19]:
!pip install ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.7 MB/s 


In [6]:
from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer
import torch

# トークナイザーとテキストエンコーダーの準備
tokenizer = CLIPTokenizer.from_pretrained(
    pretrained_model_name_or_path,
    subfolder="tokenizer",
    use_auth_token=True,
)
text_encoder = CLIPTextModel.from_pretrained(
    pretrained_model_name_or_path, subfolder="text_encoder", use_auth_token=True
)

# 学習した特徴ベクトルをCLIPに読み込み
def load_learned_embed_in_clip(learned_embeds_path, text_encoder, tokenizer, token=None):
  loaded_learned_embeds = torch.load(learned_embeds_path, map_location="cpu")
  
  # 個別のトークンと特徴ベクトル
  trained_token = list(loaded_learned_embeds.keys())[0]
  embeds = loaded_learned_embeds[trained_token]

  # text_encoderのdtypeにキャスト
  dtype = text_encoder.get_input_embeddings().weight.dtype
  embeds.to(dtype)

  # トークナイザーにトークンを追加
  token = token if token is not None else trained_token
  num_added_tokens = tokenizer.add_tokens(token)
  if num_added_tokens == 0:
    raise ValueError(f"The tokenizer already contains the token {token}.")
  
  # トークンの特徴ベクトルのサイズ変更
  text_encoder.resize_token_embeddings(len(tokenizer))
  
  # トークンのIDを取得し特徴ベクトルを割り当てる
  token_id = tokenizer.convert_tokens_to_ids(token)
  text_encoder.get_input_embeddings().weight.data[token_id] = embeds
load_learned_embed_in_clip(
    f"{downloaded_embedding_folder}/learned_embeds.bin", 
    text_encoder, 
    tokenizer)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/525k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/806 [00:00<?, ?B/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


Downloading:   0%|          | 0.00/592 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/492M [00:00<?, ?B/s]

In [8]:
from diffusers import StableDiffusionPipeline

# Stable Diffusionパイプラインの準備
pipe = StableDiffusionPipeline.from_pretrained(
    pretrained_model_name_or_path,
    torch_dtype=torch.float16,
    text_encoder=text_encoder,
    tokenizer=tokenizer,
    use_auth_token=True,
).to("cuda")

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

{'trained_betas'} was not found in config. Values will be initialized to default values.


In [10]:
from torch import autocast

# プロンプト
prompt = "a <hitokomoru-style> style mecha robot"

# 推論の実行
with autocast("cuda"):
    images = pipe(
        prompt, 
        num_inference_steps=50, 
        guidance_scale=7.5,
        ).images
images[0].save("output.png")

  0%|          | 0/51 [00:00<?, ?it/s]